# Bike Sharing Demand

###Data Understanding

In [1]:
!pip install skits

import pandas as pd
import numpy as np
import math
import pickle
from scipy.stats import kruskal, pearsonr, randint, uniform, chi2_contingency, boxcox
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, FunctionTransformer, StandardScaler, power_transform
from sklearn.linear_model import SGDClassifier
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error
from sklearn.model_selection import cross_val_score, cross_validate, TimeSeriesSplit, RandomizedSearchCV, GridSearchCV, cross_val_predict
from datetime import datetime
from statsmodels.tsa.stattools import grangercausalitytests, adfuller, kpss, acf, pacf
from collections import defaultdict, OrderedDict
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from sklearn.decomposition import PCA
from statsmodels.tsa.ar_model import AR
from skits.feature_extraction import AutoregressiveTransformer
from skits.preprocessing import ReversibleImputer
from sklearn.linear_model import LinearRegression
import xgboost as xgb

import seaborn as sb
import matplotlib.pyplot as plt 

%matplotlib inline


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# diplaying all columns without truncation in dataframes
pd.set_option('display.max_columns', 500)


#### Loading the data and visualizing and describing it

In [3]:
# read in bike sharing dataset
bike_df = pd.read_csv('/content/bike_sharing_dataset.csv')

In [4]:
# fill NAs with 0 where applicable
bike_df['holiday'] = bike_df['holiday'].fillna(0)
bike_df['holiday'] = bike_df['holiday'].astype('int')

In [5]:
# function to create seasons for dataframe
def seasons(df):
   # create a season features
    df['season_spring'] = df['Date'].apply(lambda x: 1 if '03' in x[5:7] else 1 if '04' in x[5:7] else 1 
                                                     if '05' in x[5:7] else 0)
    df['season_summer'] = df['Date'].apply(lambda x: 1 if '06' in x[5:7] else 1 if '07' in x[5:7] else 1 
                                                     if '08' in x[5:7] else 0)
    df['season_fall'] = df['Date'].apply(lambda x: 1 if '09' in x[5:7] else 1 if '10' in x[5:7] else 1 
                                                     if '11' in x[5:7] else 0)
    df['season_winter'] = df['Date'].apply(lambda x: 1 if '12' in x[5:7] else 1 if '01' in x[5:7] else 1 
                                                     if '02' in x[5:7] else 0)
    
    return df


In [6]:
### create new features for seasons
bike_df = seasons(bike_df)

In [7]:
### create new feature weekday
bike_df['date_datetime'] = bike_df['Date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
bike_df['weekday'] = bike_df['date_datetime'].apply(lambda x: x.weekday())

In [8]:
### one hot encode the feature weekday
## 0 = Monday, 1 = Tuesday, 2 = Wednesday, etc...
weekday_dummies = pd.get_dummies(bike_df['weekday'], prefix='weekday', drop_first=True)
bike_df = bike_df.join(weekday_dummies, how='left')

In [9]:
### create new feature working_day
bike_df['working_day'] = bike_df['weekday'].apply(lambda x: 0 if x > 5 or x == 0 else 1)
bike_df['working_day'] = bike_df[['holiday', 'working_day']].apply(
    lambda x: 0 if x['holiday'] == 1 else x['working_day'], axis=1)

In [10]:
# drop columns that are irrelevant
bike_df.drop(columns=['City', 'ID', 'DistrictKOR', 'DistrictENG', 
                      'Area(m2)','Bike_Time_NONZERO_SUM',	'Bike_Time_NONZERO_Mean',	
                      'Bike_Distance_NONZERO_SUM',	'Bike_Distance_NONZERO_Mean', 
                      'Trip_Count_NONZERO', 'Bike_Time_SUM', 'BIKE_TIME_MEAN', 
                      'Bike_Distance_SUM', 'BIKE_DISTANCE_MEAN'], inplace=True)

In [11]:
# create a new dataframe that encodes the weekday feature with 0 for monday through friday
# and 1 for saturday and sunday
weekend_distinct_df = bike_df.copy()
weekend_distinct_df['weekday'] = weekend_distinct_df['weekday'].apply(lambda x: 1 if (x == 6 or x == 0) else 0)

In [12]:
bike_df.drop(columns=['WinddirectM(deg)'], inplace=True)

In [13]:
# get list for all correlations between a feature and Trip_Count with different rolling means
def best_window(x, y, max_window):
    corr_temp_cust = []
    for i in range(1, max_window):
        roll_val = list(x.rolling(i).mean()[i-1:-1])
        Trip_Count_ti = list(y[i:])
        corr, p_val = pearsonr(Trip_Count_ti, roll_val)
        corr_temp_cust.append(corr)
    # get the optimal window size for rolling mean between a feature and Trip_Count
    max_val = np.argmax(corr_temp_cust)
    min_val = np.argmin(corr_temp_cust)
    opt_corr_min = corr_temp_cust[min_val]
    opt_corr_max = corr_temp_cust[max_val]
    
    results = {max_val+1: opt_corr_max, min_val+1: opt_corr_min}
    
    return results
    

In [14]:
# get list for all correlations between a feature and Trip_Count with different rolling standard deviations
def best_window_std(x, y, max_window):
    corr_temp_cust = []
    for i in range(2, max_window):
        roll_val = list(x.rolling(i).std()[i-1:-1])
        Trip_Count_ti = list(y[i:])
        corr, p_val = pearsonr(Trip_Count_ti, roll_val)
        corr_temp_cust.append(corr)
    # get the optimal window size for rolling std between a feature and Trip_Count
    max_val = np.argmax(corr_temp_cust)
    min_val = np.argmin(corr_temp_cust)
    opt_corr_min = corr_temp_cust[min_val]
    opt_corr_max = corr_temp_cust[max_val]
    
    results = {max_val+1: opt_corr_max, min_val+1: opt_corr_min}
    
    return results


In [15]:
# get the optimal window for rolling std for Trip_Count
print(best_window_std(bike_df['Trip_Count'], bike_df['Trip_Count'], 40))

# get the correlation for window size determined by Trip_Count
cust_mean = bike_df['Trip_Count'].rolling(8).std()[7:-1]
pearsonr(cust_mean, bike_df['Trip_Count'][8:])

{21: 0.7458671048146939, 1: 0.3185045110199073}


(0.5909874097580865, 0.0)

In [16]:
# get the optimal number for rolling mean for Trip_Count
print(best_window(bike_df['Trip_Count'], bike_df['Trip_Count'], 40))

# get the correlation for window size determined by Trip_Count
cust_mean = bike_df['Trip_Count'].rolling(8).mean()[7:-1]
pearsonr(cust_mean, bike_df['Trip_Count'][8:])


{7: 0.8878801976300186, 39: 0.8177747047009136}


(0.8863948653212457, 0.0)

In [17]:
# add the value from t-1
bike_df['Trip_Count_t-1'] = bike_df['Trip_Count'].shift()


In [18]:
# create series that group the mean temperature per season
temp_spring = bike_df.groupby('season_spring')['TempM(°C)'].mean().rename({1: 'Spring'})
temp_summer = bike_df.groupby('season_summer')['TempM(°C)'].mean().rename({1: 'Summer'})
temp_fall = bike_df.groupby('season_fall')['TempM(°C)'].mean().rename({1: 'Fall'})
temp_winter = bike_df.groupby('season_winter')['TempM(°C)'].mean().rename({1: 'Winter'})

# add them to one series and drop the rows with index 0
temp_seasons = temp_summer.append(temp_fall).append(temp_winter).append(temp_spring)
temp_seasons.drop(labels=[0], inplace=True)

In [19]:
# create series that groups average users per season
cust_spring = bike_df.groupby('season_spring')['Trip_Count'].mean().rename({1: 'Spring'})
cust_summer = bike_df.groupby('season_summer')['Trip_Count'].mean().rename({1: 'Summer'})
cust_fall = bike_df.groupby('season_fall')['Trip_Count'].mean().rename({1: 'Fall'})
cust_winter = bike_df.groupby('season_winter')['Trip_Count'].mean().rename({1: 'Winter'})

# add them to one series and drop the rows with index 0
cust_seasons = cust_summer.append(cust_fall).append(cust_winter).append(cust_spring)
cust_seasons.drop(labels=[0], inplace=True)

In [20]:
# get the optimal window for rolling std for temperature
print(best_window_std(bike_df['TempM(°C)'], bike_df['Trip_Count'], 40))

# get the correlation for window size determined by TempM(°C)
temp_mean = bike_df['TempM(°C)'].rolling(8).std()[7:-1]
pearsonr(temp_mean, bike_df['Trip_Count'][8:])

{1: -0.11818513201317055, 31: -0.279594519569663}


(-0.2260233053368831, 5.0153556185029124e-141)

In [21]:
# get the optimal window for rolling mean for temperature
best_window(bike_df['TempM(°C)'], bike_df['Trip_Count'], 40)


{6: 0.38647683685028433, 39: 0.3383048120182877}

In [22]:
# engineer new categorical AQ features
bike_df_cat = bike_df.copy()
bike_df_cat['PM10_good'] = bike_df_cat['PM10'].apply(lambda x: 1 if x < 30 else 0)
bike_df_cat['PM10_moderate'] = bike_df_cat['PM10'].apply(lambda x: 1 if x < 80 and x >= 31 else 0)
bike_df_cat['PM10_unhealthy'] = bike_df_cat['PM10'].apply(lambda x: 1 if x < 150 and x >= 81 else 0)
bike_df_cat['PM10_very_unhealthy'] = bike_df_cat['PM10'].apply(lambda x: 1 if x >= 151 else 0)
bike_df_cat['PM25_good'] = bike_df_cat['PM2.5'].apply(lambda x: 1 if x < 15 else 0)
bike_df_cat['PM25_moderate'] = bike_df_cat['PM2.5'].apply(lambda x: 1 if x < 35 and x >= 16 else 0)
bike_df_cat['PM25_unhealthy'] = bike_df_cat['PM2.5'].apply(lambda x: 1 if x < 75 and x >= 36 else 0)
bike_df_cat['PM25_very_unhealthy'] = bike_df_cat['PM2.5'].apply(lambda x: 1 if x >= 76 else 0)

bike_df_cat[['PM10_good', 'PM10_moderate', 'PM10_unhealthy', 'PM10_very_unhealthy', 'PM25_good', 'PM25_moderate', 'PM25_unhealthy', 'PM25_very_unhealthy']] = bike_df_cat[['PM10_good', 'PM10_moderate', 'PM10_unhealthy', 'PM10_very_unhealthy', 'PM25_good', 'PM25_moderate', 'PM25_unhealthy', 'PM25_very_unhealthy']].shift()

bike_df_cat = bike_df_cat.iloc[1:,:]

In [23]:
# get the optimal window for rolling std for RainM(mm)
print(best_window_std(bike_df['RainM(mm)'], bike_df['Trip_Count'], 40))

# get the correlation for window size determined by RainM(mm)
precip_mean = bike_df['RainM(mm)'].rolling(8).std()[7:-1]
pearsonr(precip_mean, bike_df['Trip_Count'][8:])


{38: 0.061215418070146466, 2: -0.021762566657129716}


(-0.0003705742186305601, 0.9673647401483304)

In [24]:
# get the optimal number for rolling mean window
print(best_window(bike_df['WindSpeedM(m/s)'], bike_df['Trip_Count'], 40))

# get the correlation for window size determined by wind
wind_mean = bike_df['WindSpeedM(m/s)'].rolling(8).mean()[7:-1]
pearsonr(wind_mean, bike_df['Trip_Count'][8:])[0]


{39: 0.023631717822896525, 1: -0.026152704864007892}


-0.0026884178670223413

In [25]:
# get the optimal window for rolling std for temperature
print(best_window_std(bike_df['WindSpeedM(m/s)'], bike_df['Trip_Count'], 40))

# get the correlation for window size determined by wind
wind_mean = bike_df['WindSpeedM(m/s)'].rolling(8).std()[7:-1]
pearsonr(wind_mean, bike_df['Trip_Count'][8:])


{38: -0.05679909742989022, 6: -0.11929895283492986}


(-0.11382634433058314, 1.9049735545374817e-36)

In [26]:
ENTROPYBIG_mean = bike_df['ENTROPYBIG'].rolling(8).mean()[7:-1]
ENTROPYSMALL_mean = bike_df['ENTROPYSMALL'].rolling(8).mean()[7:-1]
RESIDENTIAL110_mean = bike_df['110_RESIDENTIAL'].rolling(8).mean()[7:-1]
HOUSE111_mean = bike_df['111_HOUSE'].rolling(8).mean()[7:-1]
APT112_mean = bike_df['112_APT'].rolling(8).mean()[7:-1]
INDUSTRIAL120_mean = bike_df['120_INDUSTRIAL'].rolling(8).mean()[7:-1]
INDUSTRIAL121_mean = bike_df['121_INDUSTRIAL'].rolling(8).mean()[7:-1]
COMMERCIAL130_mean = bike_df['130_COMMERCIAL'].rolling(8).mean()[7:-1]
COMMERCIAL131_WORK_mean = bike_df['131_COMMERCIAL_WORK'].rolling(8).mean()[7:-1]
MIXEDUSE132_mean = bike_df['132_MIXEDUSE'].rolling(8).mean()[7:-1]
RECREATIONAL140_mean = bike_df['140_RECREATIONAL'].rolling(8).mean()[7:-1]
RECREATIONAL141_mean = bike_df['141_RECREATIONAL'].rolling(8).mean()[7:-1]
TRANSIT150_mean = bike_df['150_TRANSIT'].rolling(8).mean()[7:-1]
PUBLIC160_mean = bike_df['160_PUBLIC'].rolling(8).mean()[7:-1]
GREENINFRA_mean = bike_df['GREENINFRA'].rolling(8).mean()[7:-1]
POPDensity_mean = bike_df['POPDensity(p/㎢)'].rolling(8).mean()[7:-1]
POPGender_mean = bike_df['POPGender(Male/Female)'].rolling(8).mean()[7:-1]
Age_under24_mean = bike_df['Age_under24'].rolling(8).mean()[7:-1]
PM10_mean = bike_df['PM10'].rolling(8).mean()[7:-1]
PM25_mean = bike_df['PM2.5'].rolling(8).mean()[7:-1]
COVID_mean = bike_df['COVID'].rolling(8).mean()[7:-1]
Metro_station_mean = bike_df['Metro_station'].rolling(8).mean()[7:-1]
Metrotrip18_IN_mean = bike_df['18Metrotrip_IN'].rolling(8).mean()[7:-1]
Metrotrip18_OUT_mean = bike_df['18Metrotrip_OUT'].rolling(8).mean()[7:-1]
Metrotrip_IN_mean = bike_df['Metrotrip_IN'].rolling(8).mean()[7:-1]
Metrotrip_OUT_mean = bike_df['Metrotrip_OUT'].rolling(8).mean()[7:-1]
Bus_station_mean = bike_df['Bus_station'].rolling(8).mean()[7:-1]
BusINCount_mean = bike_df['BusINCount'].rolling(8).mean()[7:-1]
BusOUTCount_mean = bike_df['BusOUTCount'].rolling(8).mean()[7:-1]
Bike_station_mean = bike_df['Bike_station'].rolling(8).mean()[7:-1]
Bike_stand_mean = bike_df['Bike_stand'].rolling(8).mean()[7:-1]
BikeRoadCount_mean = bike_df['BikeRoadCount(개)'].rolling(8).mean()[7:-1]
BikeRoadDistance_mean = bike_df['BikeRoadDistance(km)'].rolling(8).mean()[7:-1]

In [27]:
# code based on implementation on https://www.analyticsvidhya.com/blog/2018/09/non-stationary-time-series-python/
def adf_test(df, col_names):
    '''
    Function to perform Augmented Dickey-Fuller test on selected timeseries
    Args: df = dataframe with timeseries to be tested
          col_names = list of names of the timeseries to be tested
    Returns: None
    '''
    for name in col_names:
        print ('Results of Augmented Dickey-Fuller Test for {}'.format(name))
        result_test = adfuller(df[name], autolag='AIC')
        result_output = pd.Series(result_test[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
        for key, val in result_test[4].items():
            result_output['Critical Value (%s)'%key] = val
        print (result_output)


In [28]:
# create the features that need to be tested
# Trip_Count_t-1 was already added to the dataframe

testing_feat = ['Trip_Count', 'ENTROPYBIG',	'ENTROPYSMALL',	'110_RESIDENTIAL',	'111_HOUSE',	'112_APT',	'120_INDUSTRIAL',	'121_INDUSTRIAL',	'130_COMMERCIAL',	'131_COMMERCIAL_WORK',	'132_MIXEDUSE',	'140_RECREATIONAL',	'141_RECREATIONAL',	'150_TRANSIT',	'160_PUBLIC',	'GREENINFRA',	'POPDensity(p/㎢)',	'POPGender(Male/Female)',	'Age_under24',	'TempM(°C)',	'WindSpeedM(m/s)',	'RainM(mm)',	'PM10',	'PM2.5',	'COVID',	'Metro_station',	'18Metrotrip_IN',	'18Metrotrip_OUT',	'Metrotrip_IN',	'Metrotrip_OUT',	'Bus_station',	'BusINCount',	'BusOUTCount',	'Bike_station',	'Bike_stand',	'BikeRoadCount(개)',	'BikeRoadDistance(km)']

testing_df = pd.DataFrame()

for col in testing_feat:
    col_mean = bike_df[col].rolling(16).mean()[15:-1]
    col_std = bike_df[col].rolling(16).std()[15:-1]
    testing_df[col+'_mean16'] = col_mean.values
    testing_df[col+'_std16'] = col_std.values


In [29]:
# adf test for Trip_Count_t-1
temp_cust_1 = bike_df['Trip_Count_t-1'].fillna(0)
bike_df_temp = pd.DataFrame(temp_cust_1, columns=['Trip_Count_t-1'])
adf_test(bike_df_temp, ['Trip_Count_t-1'])

Results of Augmented Dickey-Fuller Test for Trip_Count_t-1
Test Statistic                -7.513128e+00
p-value                        3.967003e-11
#Lags Used                     4.000000e+01
Number of Observations Used    1.215900e+04
Critical Value (1%)           -3.430888e+00
Critical Value (5%)           -2.861778e+00
Critical Value (10%)          -2.566897e+00
dtype: float64


In [30]:
# adf test for Trip_Count
adf_test(bike_df, ['Trip_Count'])

Results of Augmented Dickey-Fuller Test for Trip_Count
Test Statistic                -7.513455e+00
p-value                        3.959547e-11
#Lags Used                     4.000000e+01
Number of Observations Used    1.215900e+04
Critical Value (1%)           -3.430888e+00
Critical Value (5%)           -2.861778e+00
Critical Value (10%)          -2.566897e+00
dtype: float64


In [31]:
# code based on implementation on https://www.analyticsvidhya.com/blog/2018/09/non-stationary-time-series-python/
def kpss_test(df, col_names):
    '''
    Function to perform KPSS test on selected timeseries
    Args: df = dataframe with timeseries to be tested
          col_names = list of names of the timeseries to be tested
    Returns: None
    '''
    for name in col_names:
        print ('Results of KPSS Test for {}'.format(name))
        result_test = kpss(df[name], regression='c', lags='legacy')
        result_output = pd.Series(result_test[0:3], index=['Test Statistic','p-value','Lags Used'])
        for key, val in result_test[3].items():
            result_output['Critical Value (%s)'%key] = val
        print (result_output)

In [32]:
# kpss test for Trip_Count_t-1
kpss_test(bike_df_temp, ['Trip_Count_t-1'])

# kpss test for Trip_Count
kpss_test(bike_df, ['Trip_Count'])


Results of KPSS Test for Trip_Count_t-1
Test Statistic            3.787181
p-value                   0.010000
Lags Used                40.000000
Critical Value (10%)      0.347000
Critical Value (5%)       0.463000
Critical Value (2.5%)     0.574000
Critical Value (1%)       0.739000
dtype: float64
Results of KPSS Test for Trip_Count
Test Statistic            3.780879
p-value                   0.010000
Lags Used                40.000000
Critical Value (10%)      0.347000
Critical Value (5%)       0.463000
Critical Value (2.5%)     0.574000
Critical Value (1%)       0.739000
dtype: float64


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/stattools.py:1709: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/stattools.py:1709: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)


In [33]:
# Trying to lessen trends (make it stationary) via log

not_stationary =['ENTROPYBIG_mean16',	'ENTROPYSMALL_mean16',	'110_RESIDENTIAL_mean16',	
                 '111_HOUSE_mean16',	'112_APT_mean16',	'120_INDUSTRIAL_mean16',	'121_INDUSTRIAL_mean16',	
                 '130_COMMERCIAL_mean16',	'131_COMMERCIAL_WORK_mean16',	'132_MIXEDUSE_mean16',	'140_RECREATIONAL_mean16',	
                 '141_RECREATIONAL_mean16',	'150_TRANSIT_mean16',	'160_PUBLIC_mean16',	'GREENINFRA_mean16',	
                 'POPDensity(p/㎢)_mean16',	'POPGender(Male/Female)_mean16',	'Age_under24_mean16',	'WindSpeedM(m/s)_mean16',	
                 'Metro_station_mean16',	'18Metrotrip_IN_mean16',	'18Metrotrip_OUT_mean16',	'Metrotrip_IN_mean16',	
                 'Metrotrip_OUT_mean16',	'Bus_station_mean16',	'Bike_station_mean16',	'Bike_stand_mean16',	
                 'BikeRoadCount(개)_mean16',	'BikeRoadDistance(km)_mean16',	'Trip_Count_mean16',	'Trip_Count_t-1_mean16',	
                 'WindSpeedM(m/s)_std16',	'18Metrotrip_IN_std16',	'18Metrotrip_OUT_std16',	'Metrotrip_IN_std16',	'Metrotrip_OUT_std16']

bike_df_temp['Trip_Count_t-1_log'] = [np.log1p(x+1) for x in bike_df_temp['Trip_Count_t-1']]
testing_df['ENTROPYBIG_mean16_log'] = [np.log1p(x+1) for x in testing_df['ENTROPYBIG_mean16']]
testing_df['ENTROPYSMALL_mean16_log'] = [np.log1p(x+1) for x in testing_df['ENTROPYSMALL_mean16']]
testing_df['110_RESIDENTIAL_mean16_log'] = [np.log1p(x+1) for x in testing_df['110_RESIDENTIAL_mean16']]
testing_df['111_HOUSE_mean16_log'] = [np.log1p(x+1) for x in testing_df['111_HOUSE_mean16']]
testing_df['112_APT_mean16_log'] = [np.log1p(x+1) for x in testing_df['112_APT_mean16']]
testing_df['120_INDUSTRIAL_mean16_log'] = [np.log1p(x+1) for x in testing_df['120_INDUSTRIAL_mean16']]
testing_df['121_INDUSTRIAL_mean16_log'] = [np.log1p(x+1) for x in testing_df['121_INDUSTRIAL_mean16']]
testing_df['130_COMMERCIAL_mean16_log'] = [np.log1p(x+1) for x in testing_df['130_COMMERCIAL_mean16']]
testing_df['131_COMMERCIAL_WORK_mean16_log'] = [np.log1p(x+1) for x in testing_df['131_COMMERCIAL_WORK_mean16']]
testing_df['132_MIXEDUSE_mean16_log'] = [np.log1p(x+1) for x in testing_df['132_MIXEDUSE_mean16']]
testing_df['140_RECREATIONAL_mean16_log'] = [np.log1p(x+1) for x in testing_df['140_RECREATIONAL_mean16']]
testing_df['141_RECREATIONAL_mean16_log'] = [np.log1p(x+1) for x in testing_df['141_RECREATIONAL_mean16']]
testing_df['150_TRANSIT_mean16_log'] = [np.log1p(x+1) for x in testing_df['150_TRANSIT_mean16']]
testing_df['160_PUBLIC_mean16_log'] = [np.log1p(x+1) for x in testing_df['160_PUBLIC_mean16']]
testing_df['GREENINFRA_mean16_log'] = [np.log1p(x+1) for x in testing_df['GREENINFRA_mean16']]
testing_df['POPDensity(p/㎢)_mean16_log'] = [np.log1p(x+1) for x in testing_df['POPDensity(p/㎢)_mean16']]
testing_df['POPGender(Male/Female)_mean16_log'] = [np.log1p(x+1) for x in testing_df['POPGender(Male/Female)_mean16']]
testing_df['Age_under24_mean16_log'] = [np.log1p(x+1) for x in testing_df['Age_under24_mean16']]
testing_df['WindSpeedM(m/s)_mean16_log'] = [np.log1p(x+1) for x in testing_df['WindSpeedM(m/s)_mean16']]
testing_df['Metro_station_mean16_log'] = [np.log1p(x+1) for x in testing_df['Metro_station_mean16']]
testing_df['18Metrotrip_IN_mean16_log'] = [np.log1p(x+1) for x in testing_df['18Metrotrip_IN_mean16']]
testing_df['18Metrotrip_OUT_mean16_log'] = [np.log1p(x+1) for x in testing_df['18Metrotrip_OUT_mean16']]
testing_df['Metrotrip_IN_mean16_log'] = [np.log1p(x+1) for x in testing_df['Metrotrip_IN_mean16']]
testing_df['Metrotrip_OUT_mean16_log'] = [np.log1p(x+1) for x in testing_df['Metrotrip_OUT_mean16']]
testing_df['Bus_station_mean16_log'] = [np.log1p(x+1) for x in testing_df['Bus_station_mean16']]
testing_df['Bike_station_mean16_log'] = [np.log1p(x+1) for x in testing_df['Bike_station_mean16']]
testing_df['Bike_stand_mean16_log'] = [np.log1p(x+1) for x in testing_df['Bike_stand_mean16']]
testing_df['BikeRoadCount(개)_mean16_log'] = [np.log1p(x+1) for x in testing_df['BikeRoadCount(개)_mean16']]
testing_df['BikeRoadDistance(km)_mean16_log'] = [np.log1p(x+1) for x in testing_df['BikeRoadDistance(km)_mean16']]
testing_df['Trip_Count_mean16_log'] = [np.log1p(x+1) for x in testing_df['Trip_Count_mean16']]
testing_df['WindSpeedM(m/s)_std16_log'] = [np.log1p(x+1) for x in testing_df['WindSpeedM(m/s)_std16']]
testing_df['18Metrotrip_IN_std16_log'] = [np.log1p(x+1) for x in testing_df['18Metrotrip_IN_std16']]
testing_df['18Metrotrip_OUT_std16_log'] = [np.log1p(x+1) for x in testing_df['18Metrotrip_OUT_std16']]
testing_df['Metrotrip_IN_std16_log'] = [np.log1p(x+1) for x in testing_df['Metrotrip_IN_std16']]
testing_df['Metrotrip_OUT_std16_log'] = [np.log1p(x+1) for x in testing_df['Metrotrip_OUT_std16']]


In [ ]:
# applying differencing to remove trend from Trip_Count
bike_df_check = bike_df[['Trip_Count']]
bike_df_check['Trip_Count_diff'] = bike_df_check['Trip_Count'] - bike_df_check['Trip_Count'].shift()
bike_df_check = bike_df_check.iloc[1:,]

In [ ]:
# kpss test for Trip_Count
kpss_test(testing_df, ['Trip_Count_mean16_log'])
kpss_test(bike_df_temp, ['Trip_Count_t-1_log'])

# adf test for Trip_Count
adf_test(testing_df, ['Trip_Count_mean16_log'])
adf_test(bike_df_temp, ['Trip_Count_t-1_log'])
adf_test(bike_df_check, ['Trip_Count_diff'])


In [36]:
# should keep this before the cleaning function has been created
bike_df.drop(columns=['COVID/100000', 'weekday'], axis=1, inplace=True)

In [37]:
# drop the timestamp variable
bike_df.drop(columns=['date_datetime'], inplace=True)

In [38]:
# specify the window for rolling values
window = 8

In [39]:
# creating rolling values
new_feat = ['Trip_Count', 'ENTROPYBIG',	'ENTROPYSMALL',	'110_RESIDENTIAL',	'111_HOUSE',	'112_APT',	'120_INDUSTRIAL',	'121_INDUSTRIAL',	'130_COMMERCIAL',	'131_COMMERCIAL_WORK',	'132_MIXEDUSE',	'140_RECREATIONAL',	'141_RECREATIONAL',	'150_TRANSIT',	'160_PUBLIC',	'GREENINFRA',	'POPDensity(p/㎢)',	'POPGender(Male/Female)',	'Age_under24',	'TempM(°C)',	'WindSpeedM(m/s)',	'RainM(mm)',	'PM10',	'PM2.5',	'COVID',	'Metro_station',	'18Metrotrip_IN',	'18Metrotrip_OUT',	'Metrotrip_IN',	'Metrotrip_OUT',	'Bus_station',	'BusINCount',	'BusOUTCount',	'Bike_station',	'Bike_stand',	'BikeRoadCount(개)',	'BikeRoadDistance(km)']

temp_df = pd.DataFrame()

for col in new_feat:
    col_mean = bike_df[col].rolling(window).mean()[(window-1):-1]
    col_std = bike_df[col].rolling(window).std()[(window-1):-1]
    temp_df[col+'_mean'+str(window)] = col_mean.values
    temp_df[col+'_std'+str(window)] = col_std.values

In [40]:
# remember to remove the first row from 16 rows from Trip_Count (target label)
new_bike_df = bike_df.iloc[window:,:]
bike_df_cat = bike_df_cat.iloc[window:,:]
new_bike_df.reset_index(drop=True, inplace=True)
bike_df_cat.reset_index(drop=True, inplace=True)

In [41]:
# merging both dataframes with features
final_bike_df = new_bike_df.join(temp_df, how='left')
final_bike_df = final_bike_df.merge(bike_df_cat, how='left')

In [56]:
# assigning X and y
y = final_bike_df['Trip_Count']
X = final_bike_df.drop(columns=['Trip_Count'])

final_bike_df['Date'] = pd.to_datetime(final_bike_df['Date'], format='%Y-%m-%d').astype(int)
final_bike_df['Date'] = final_bike_df['Date'].apply(lambda x: str(x))
final_bike_df['date_datetime'] = pd.to_datetime(final_bike_df['date_datetime'], format='%Y-%m-%d').astype(int)
final_bike_df['date_datetime'] = final_bike_df['date_datetime'].apply(lambda x: str(x))

,Date,ENTROPYBIG,ENTROPYSMALL,110_RESIDENTIAL,111_HOUSE,112_APT,120_INDUSTRIAL,121_INDUSTRIAL,130_COMMERCIAL,131_COMMERCIAL_WORK,132_MIXEDUSE,140_RECREATIONAL,141_RECREATIONAL,150_TRANSIT,160_PUBLIC,GREENINFRA,POPDensity(p/㎢),POPGender(Male/Female),Age_under24,TempM(°C),WindSpeedM(m/s),RainM(mm),PM10,PM2.5,COVID,Metro_station,18Metrotrip_IN,18Metrotrip_OUT,Metrotrip_IN,Metrotrip_OUT,Bus_station,BusINCount,BusOUTCount,Bike_station,Bike_stand,BikeRoadCount(개),BikeRoadDistance(km),SKTPOP,holiday,season_spring,season_summer,season_fall,season_winter,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,working_day,Trip_Count_t-1,Trip_Count_mean8,Trip_Count_std8,ENTROPYBIG_mean8,ENTROPYBIG_std8,ENTROPYSMALL_mean8,ENTROPYSMALL_std8,110_RESIDENTIAL_mean8,110_RESIDENTIAL_std8,111_HOUSE_mean8,111_HOUSE_std8,112_APT_mean8,112_APT_std8,120_INDUSTRIAL_mean8,120_INDUSTRIAL_std8,121_INDUSTRIAL_mean8,121_INDUSTRIAL_std8,130_COMMERCIAL_mean8,130_COMMERCIAL_std8,131_COMMERCIAL_WORK_mean8,131_COMMERCIAL_WORK_std8,132_MIXEDUSE_mean8,132_MIXEDUSE_std8,140_RECREATIONAL_mean8,140_RECREATIONAL_std8,141_RECREATIONAL_mean8,141_RECREATIONAL_std8,150_TRANSIT_mean8,150_TRANSIT_std8,160_PUBLIC_mean8,160_PUBLIC_std8,GREENINFRA_mean8,GREENINFRA_std8,POPDensity(p/㎢)_mean8,POPDensity(p/㎢)_std8,POPGender(Male/Female)_mean8,POPGender(Male/Female)_std8,Age_under24_mean8,Age_under24_std8,TempM(°C)_mean8,TempM(°C)_std8,WindSpeedM(m/s)_mean8,WindSpeedM(m/s)_std8,RainM(mm)_mean8,RainM(mm)_std8,PM10_mean8,PM10_std8,PM2.5_mean8,PM2.5_std8,COVID_mean8,COVID_std8,Metro_station_mean8,Metro_station_std8,18Metrotrip_IN_mean8,18Metrotrip_IN_std8,18Metrotrip_OUT_mean8,18Metrotrip_OUT_std8,Metrotrip_IN_mean8,Metrotrip_IN_std8,Metrotrip_OUT_mean8,Metrotrip_OUT_std8,Bus_station_mean8,Bus_station_std8,BusINCount_mean8,BusINCount_std8,BusOUTCount_mean8,BusOUTCount_std8,Bike_station_mean8,Bike_station_std8,Bike_stand_mean8,Bike_stand_std8,BikeRoadCount(개)_mean8,BikeRoadCount(개)_std8,BikeRoadDistance(km)_mean8,BikeRoadDistance(km)_std8,COVID/100000,date_datetime,weekday,PM10_good,PM10_moderate,PM10_unhealthy,PM10_very_unhealthy,PM25_good,PM25_moderate,PM25_unhealthy,PM25_very_unhealthy
0,1552089600000000000,0.568,0.658,26.798993,5.660696,5.374966,0.110598,0.144978,4.704451,4.557102,0.132349,0.68001,0.845942,9.150575,3.376849,48.116125,16245,94.36,0.218147,5.645833,0.837500,0.000000,37.0,26.0,0,8,65463,65411,90049,85338,390,123134,118254,33,396,4,12.3,5765180,0,1,0,0,0,0,0,0,0,1,0,1,465.0,395.625,55.538757,0.568,0.0,0.658,0.0,26.798993,0.0,5.660696,0.0,5.374966,0.0,0.110598,0.0,0.144978,0.0,4.704451,0.0,4.557102,0.0,0.132349,0.0,0.68001,0.0,0.845942,0.0,9.150575,0.0,3.376849,0.0,48.116125,0.0,16245.0,0.0,94.36,0.0,0.218147,0.0,7.414062,1.344689,1.141146,0.193706,0.000000,0.000000,108.500,46.213789,76.875,36.290249,0.0,0.0,8.0,0.0,71326.375,15551.801659,71975.500,15500.323951,95244.750,18566.365463,91519.000,17933.040767,390.0,0.0,136703.250,30918.805871,132333.50,29409.877481,33.0,0.0,396.0,0.0,4.0,0.0,12.3,0.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1552176000000000000,0.568,0.658,26.798993,5.660696,5.374966,0.110598,0.144978,4.704451,4.557102,0.132349,0.68001,0.845942,9.150575,3.376849,48.116125,16245,94.36,0.218147,9.050000,1.120833,0.000000,53.0,36.0,0,8,47242,48252,66210,63574,390,90996,88587,33,396,4,12.3,6066380,0,1,0,0,0,0,0,0,0,0,1,0,525.0,419.500,65.434809,0.568,0.0,0.658,0.0,26.798993,0.0,5.660696,0.0,5.374966,0.0,0.110598,0.0,0.144978,0.0,4.704451,0.0,4.557102,0.0,0.132349,0.0,0.68001,0.0,0.845942,0.0,9.150575,0.0,3.376849,0.0,48.116125,0.0,16245.0,0.0,94.36,0.0,0.218147,0.0,7.356771,1.416327,1.088542,0.213469,0.000000,0.000000,100.375,52.768869,71.250,40.566524,0.0,0.0,8.0,0.0,72530.125,14519.701536,73210.500,14354.141155,96948.000,17165.158008,93143.250,16477.587763,390.0,0.0,139357.250,28305.617089,134820.75,26856.381970,33.0,0.0,396.0,0.0,4.0,0.0,12.3,0.0,0.0,2019-03-10,6.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1552262400000000000,0.568,0.658,26.798993,5.6606

In [43]:
# creating a class that I can use in the ML pipeline that prints out the transformed data that will enter the model
class Debug(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        self.shape = X.shape
        print(self.shape)
        X_df = pd.DataFrame(X)
        print(X_df)
        # print(X_df.to_string()) # can only be print like this without running LagVars() to avoid crashing
        # what other output you want
        return X

In [44]:
# assigning X and y for the univariate naive prediction
y_naive = (final_bike_df['Trip_Count'].copy())
X_naive = y_naive.copy()
X_naive = pd.DataFrame(data=X_naive, columns=['Trip_Count'])

# getting the start time
start_time = datetime.now()

# creating y_pred
y_pred = (X_naive.shift())[1:]
# adjusting length of the actual target values
y_naive = y_naive[1:]

# get final time
end_time = datetime.now()
print('Total running time of naive predictor:', (end_time - start_time).total_seconds())

# calculating the scores for the last value method
print('RMSE:', np.sqrt(mean_squared_error(y_naive, y_pred)))
print('RMSLE:', np.sqrt(mean_squared_log_error(y_naive, y_pred)))
print('MAE:', mean_absolute_error(y_naive, y_pred))

Total running time of naive predictor: 0.000533
RMSE: 793.5107478722404
RMSLE: 0.4937500277399892
MAE: 468.14723976704124


#### Preparation code for using differenced y for predictions

In [45]:
y_log = final_bike_df[['Trip_Count']].copy()
y_log['Trip_Count'] = y_log['Trip_Count'].apply(lambda x: np.log1p(x+1))
y_shift = y_log.shift(1)
y_diff = (y_log - y_shift)[1:]
X_diff = X[1:]

y_diff.reset_index(drop=True, inplace=True)
X_diff.reset_index(drop=True, inplace=True)

X_diff['Date'] = pd.to_datetime(X_diff['Date'], format='%Y-%m-%d').astype(int)
X_diff['Date'] = X_diff['Date'].apply(lambda x: str(x))
X_diff['date_datetime'] = pd.to_datetime(X_diff['date_datetime'], format='%Y-%m-%d').astype(int)
X_diff['date_datetime'] = X_diff['date_datetime'].apply(lambda x: str(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [46]:
# initializing the model which is a Random Forest model and uses default hyperparameters
model_rf_diff = RandomForestRegressor(random_state=42)

In [47]:
# creating and fitting the ML pipeline
trend_features = ['Trip_Count_mean'+str(window), 'Trip_Count_std'+str(window), 'Trip_Count_t-1']

preprocessor = ColumnTransformer([
    ('trend_diff', FunctionTransformer(np.log1p, validate=False), trend_features),
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    #('debug', Debug()) # I have commented this out because it will hinder the execution of this pipeline
    ('model', model_rf_diff)
])

pipeline_rf_diff = pipeline.fit(X_diff, y_diff)


/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


In [48]:
# creating a timeseries split of the datasets
time_split = TimeSeriesSplit(n_splits=10)

# doing cross validation on the chunks of data and calculating scores
scores_rf_diff = cross_validate(pipeline_rf_diff, X_diff, y_diff, cv=time_split,
                         scoring=['neg_mean_squared_error', 'neg_mean_absolute_error'],
                         return_train_score=True, n_jobs=-1)


In [49]:
# root mean squared error
print('Average RMSE train data:', 
      sum([np.sqrt(-1 * x) for x in scores_rf_diff['train_neg_mean_squared_error']])/len(scores_rf_diff['train_neg_mean_squared_error']))
print('Average RMSE test data:', 
      sum([np.sqrt(-1 * x) for x in scores_rf_diff['test_neg_mean_squared_error']])/len(scores_rf_diff['test_neg_mean_squared_error']))

# mean absolute error
print('Average MAE train data:', 
      sum([(-1 * x) for x in scores_rf_diff['train_neg_mean_absolute_error']])/len(scores_rf_diff['train_neg_mean_absolute_error']))
print('Average MAE test data:', 
      sum([(-1 * x) for x in scores_rf_diff['test_neg_mean_absolute_error']])/len(scores_rf_diff['test_neg_mean_absolute_error']))


Average RMSE train data: 0.07879490507231
Average RMSE test data: 0.21685512068445348
Average MAE train data: 0.0486483235844493
Average MAE test data: 0.1434120534800168


In [50]:
# getting indices for all folds from timeseriessplit
X_train_over = defaultdict()  
X_test_over = defaultdict()
y_train_over = defaultdict()
y_test_over = defaultdict()

y_test_index = []
y_train_index = []

count = 0

for train_index, test_index in time_split.split(X_diff):
    print("TRAIN:", train_index, "TEST:", test_index)
    
    y_test_index.append(test_index)
    y_train_index.append(train_index)
    
    X_train, X_test = X_diff.iloc[train_index], X_diff.iloc[test_index]
    y_train, y_test = (np.array(y_diff))[train_index], (np.array(y_diff))[test_index]
    X_train_over[count] = X_train
    X_test_over[count] = X_test
    y_train_over[count] = y_train
    y_test_over[count] = y_test
    
    count += 1

TRAIN: [   0    1    2 ... 1108 1109 1110] TEST: [1111 1112 1113 ... 2216 2217 2218]
TRAIN: [   0    1    2 ... 2216 2217 2218] TEST: [2219 2220 2221 ... 3324 3325 3326]
TRAIN: [   0    1    2 ... 3324 3325 3326] TEST: [3327 3328 3329 ... 4432 4433 4434]
TRAIN: [   0    1    2 ... 4432 4433 4434] TEST: [4435 4436 4437 ... 5540 5541 5542]
TRAIN: [   0    1    2 ... 5540 5541 5542] TEST: [5543 5544 5545 ... 6648 6649 6650]
TRAIN: [   0    1    2 ... 6648 6649 6650] TEST: [6651 6652 6653 ... 7756 7757 7758]
TRAIN: [   0    1    2 ... 7756 7757 7758] TEST: [7759 7760 7761 ... 8864 8865 8866]
TRAIN: [   0    1    2 ... 8864 8865 8866] TEST: [8867 8868 8869 ... 9972 9973 9974]
TRAIN: [   0    1    2 ... 9972 9973 9974] TEST: [ 9975  9976  9977 ... 11080 11081 11082]
TRAIN: [    0     1     2 ... 11080 11081 11082] TEST: [11083 11084 11085 ... 12188 12189 12190]


In [51]:
def split_predict_test(X_train, y_train, X_test, y_test, split):
    
    pipeline_part = pipeline.fit(X_train[split], y_train[split].ravel())
    prediction_test = pipeline_part.predict(X_test[split])
    #print('transformed RMSE:', np.sqrt(mean_squared_error(y_test[split], prediction_test)))
        
    # what are the predictions in absolute terms
    y_test_pred_log = prediction_test + y_log.iloc[y_test_index[split]]['Trip_Count']
    y_test_pred = np.exp(y_test_pred_log) - 2

    # what are the actual values in absolute terms
    y_test_true_log = y_test[split] + y_log.iloc[y_test_index[split]]
    y_test_true = np.exp(y_test_true_log) - 2 # minus 2 because we added 1 each when doing the log function earlier

    # what is the rmse, rmsle and mae
    #print('actual RMSE:', np.sqrt(mean_squared_error(y_test_true, y_test_pred)))
    #print('actual RMSLE:', np.sqrt(mean_squared_log_error(y_test_true, y_test_pred)))
    #print('actual MAE:', mean_absolute_error(y_test_true, y_test_pred))
    
    rmse = np.sqrt(mean_squared_error(y_test_true, y_test_pred))
    rmsle = np.sqrt(mean_squared_log_error(y_test_true, y_test_pred))
    mae = mean_absolute_error(y_test_true, y_test_pred)
    
    list_scores = []
    list_scores.extend([rmse, rmsle, mae])
    
    return list_scores


In [52]:
def split_predict_train(X_train, y_train, X_test, y_test, split):
    
    pipeline_part = pipeline.fit(X_train[split], y_train[split].ravel())
    prediction_train = pipeline_part.predict(X_train[split])
    #print('transformed RMSE:', np.sqrt(mean_squared_error(y_test[split], prediction_test)))
        
    # what are the predictions in absolute terms
    y_train_pred_log = prediction_train + y_log.iloc[y_train_index[split]]['Trip_Count']
    y_train_pred = np.exp(y_train_pred_log) - 2

    # what are the actual values in absolute terms
    y_train_true_log = y_train[split] + y_log.iloc[y_train_index[split]]
    y_train_true = np.exp(y_train_true_log) - 2 # minus 2 because we added 1 each when doing the log function earlier

    # what is the rmse, rmsle and mae
    #print('actual RMSE:', np.sqrt(mean_squared_error(y_test_true, y_test_pred)))
    #print('actual RMSLE:', np.sqrt(mean_squared_log_error(y_test_true, y_test_pred)))
    #print('actual MAE:', mean_absolute_error(y_test_true, y_test_pred))
    
    rmse = np.sqrt(mean_squared_error(y_train_true, y_train_pred))
    rmsle = np.sqrt(mean_squared_log_error(y_train_true, y_train_pred))
    mae = mean_absolute_error(y_train_true, y_train_pred)
    
    list_scores = []
    list_scores.extend([rmse, rmsle, mae])
    
    return list_scores

#### 4.4. Random Forests

##### doing run through with untransformed y

In [53]:
# initializing the model which is a Random Forest model and uses default hyperparameters
model_rf = RandomForestRegressor(bootstrap=False, random_state=15)


In [54]:
# creating and fitting the ML pipeline
trend_features = ['Trip_Count_mean'+str(window), 'Trip_Count_std'+str(window), 'Trip_Count_t-1']

preprocessor = ColumnTransformer([
    ('trend_diff', FunctionTransformer(np.log1p, validate=False), trend_features),
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    #('debug', Debug()) # I have commented this out because it will hinder the execution of this pipeline
    ('model', model_rf)
])

pipeline_rf = pipeline.fit(X, y)


ValueError: ignored

In [ ]:
# creating a timeseries split of the datasets
time_split = TimeSeriesSplit(n_splits=10)

# doing cross validation on the chunks of data and calculating scores
scores_rf = cross_validate(pipeline_rf, X, y, cv=time_split,
                         scoring=['neg_mean_squared_error', 'neg_mean_absolute_error',
                                  'neg_mean_squared_log_error'],
                         return_train_score=True, n_jobs=-1)


In [ ]:
# root mean squared error
print('Random Forests: Average RMSE train data:', 
      sum([np.sqrt(-1 * x) for x in scores_rf['train_neg_mean_squared_error']])/len(scores_rf['train_neg_mean_squared_error']))
print('Random Forests: Average RMSE test data:', 
      sum([np.sqrt(-1 * x) for x in scores_rf['test_neg_mean_squared_error']])/len(scores_rf['test_neg_mean_squared_error']))

# mean absolute error
print('Random Forests: Average MAE train data:', 
      sum([(-1 * x) for x in scores_rf['train_neg_mean_absolute_error']])/len(scores_rf['train_neg_mean_absolute_error']))
print('Random Forests: Average MAE test data:', 
      sum([(-1 * x) for x in scores_rf['test_neg_mean_absolute_error']])/len(scores_rf['test_neg_mean_absolute_error']))

# root mean squared log error
print('Random Forests: Average RMSLE train data:', 
      sum([(-1 * x) for x in scores_rf['train_neg_mean_squared_log_error']])/len(scores_rf['train_neg_mean_squared_log_error']))
print('Random Forests: Average RMSLE test data:', 
      sum([(-1 * x) for x in scores_rf['test_neg_mean_squared_log_error']])/len(scores_rf['test_neg_mean_squared_log_error']))


I will use randomizedsearch and gridsearch to tune my hyperparameters for the Random Forest model.

In [ ]:
# number of trees in random forest
n_estimators = randint(1, 2000)
# maximum number of features included in the model
max_features = randint(1, 20)
# maximum number of levels in tree
max_depth = randint(1,10)
# minimum number of samples required to split a node
min_samples_leaf = randint(1, 10)

# create the random grid
random_grid_rf = {'model__n_estimators': n_estimators,
                   'model__max_depth': max_depth,
                   'model__min_samples_leaf': min_samples_leaf,
                   'model__max_features': max_features}


I'm using a try and except argument to load an already tuned model or, if none is available, to run randomsearch.

In [ ]:
# check the start time
start_time = datetime.now()
print(start_time)

# instantiate and fit the randomizedsearch class to the random parameters
rs_rf = RandomizedSearchCV(pipeline_rf, 
                        param_distributions=random_grid_rf, 
                        scoring='neg_mean_squared_error', 
                        n_jobs=-1,
                        cv=time_split,
                        n_iter = 5,
                        verbose=10,
                        random_state=49)
rs_rf = rs_rf.fit(X, y)

# print the total running time
end_time = datetime.now()
print('Total running time:', end_time-start_time)


In [ ]:
# Saving the best RandomForest model
pickle.dump(rs_rf.best_estimator_, open('randomforest.sav', 'wb'))

# change the keys of the best_params dictionary to allow it to be used for the final model
fix_best_params_rf = {key[7:]: val for key, val in rs_rf.best_params_.items()}

print(fix_best_params_rf)

# fit the randomforestregressor with the best_params as hyperparameters
model_rf_rs = RandomForestRegressor(**fix_best_params_rf, bootstrap=False)


In [ ]:
# creating and fitting the ML pipeline
trend_features = ['Trip_Count_mean'+str(window), 'Trip_Count_std'+str(window), 'Trip_Count_t-1']

preprocessor = ColumnTransformer([
    ('trend_diff', FunctionTransformer(np.log1p, validate=False), trend_features),
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    #('debug', Debug()) # I have commented this out because it will hinder the execution of this pipeline
    ('model', model_rf_rs)
])

# fitting x and y to pipeline
pipeline_rf_rs = pipeline.fit(X, y)


In [ ]:
# doing cross validation on the chunks of data and calculating scores
scores_rf = cross_validate(pipeline_rf_rs, X, y, cv=time_split,
                         scoring=['neg_mean_squared_error', 'neg_mean_absolute_error',
                                  'neg_mean_squared_log_error'],
                         return_train_score=True, n_jobs=-1)


In [ ]:
# root mean squared error
print('Random Forests: Average RMSE train data:', 
      sum([np.sqrt(-1 * x) for x in scores_rf['train_neg_mean_squared_error']])/len(scores_rf['train_neg_mean_squared_error']))
print('Random Forests: Average RMSE test data:', 
      sum([np.sqrt(-1 * x) for x in scores_rf['test_neg_mean_squared_error']])/len(scores_rf['test_neg_mean_squared_error']))

# absolute mean error
print('Random Forests: Average MAE train data:', 
      sum([(-1 * x) for x in scores_rf['train_neg_mean_absolute_error']])/len(scores_rf['train_neg_mean_absolute_error']))
print('Random Forests: Average MAE test data:', 
      sum([(-1 * x) for x in scores_rf['test_neg_mean_absolute_error']])/len(scores_rf['test_neg_mean_absolute_error']))

# root mean squared log error
print('Random Forests: Average RMSLE train data:', 
      sum([(-1 * x) for x in scores_rf['train_neg_mean_squared_log_error']])/len(scores_rf['train_neg_mean_squared_log_error']))
print('Random Forests: Average RMSLE test data:', 
      sum([(-1 * x) for x in scores_rf['test_neg_mean_squared_log_error']])/len(scores_rf['test_neg_mean_squared_log_error']))


In [ ]:
# assign lists of parameters to be used in gridsearch
param_grid_rf = {'model__max_depth': [8, 9],
                 'model__max_features': [18, 19],
                 'model__min_samples_leaf': [5, 6],
                 'model__n_estimators': [1926, 1930]
}


In [ ]:
# use gridsearch to search around the randomizedsearch best parameters and further improve the model
gs_rf = GridSearchCV(pipeline_rf, 
                  param_grid=param_grid_rf, 
                  scoring='neg_mean_squared_error', 
                  verbose = 10,
                  n_jobs=-1, 
                  cv=time_split)
gs_rf = gs_rf.fit(X, y)


In [ ]:
# Saving the best RandomForest model
pickle.dump(gs_rf.best_estimator_, open('randomforest.sav', 'wb'))

# change the keys of the best_params dictionary to allow it to be used for the final model
fix_best_params_rf = {key[7:]: val for key, val in gs_rf.best_params_.items()}

print(fix_best_params_rf)

# fit the randomforestregressor with the best_params as hyperparameters
model_rf_tuned = RandomForestRegressor(**fix_best_params_rf, bootstrap=False)


In [ ]:
# creating and fitting the ML pipeline
trend_features = ['Trip_Count_mean'+str(window), 'Trip_Count_std'+str(window), 'Trip_Count_t-1']

preprocessor = ColumnTransformer([
    ('trend_diff', FunctionTransformer(np.log1p, validate=False), trend_features),
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    #('debug', Debug()) # I have commented this out because it will hinder the execution of this pipeline
    ('model', model_rf_tuned)
])

# fitting x and y to pipeline
pipeline_rf_tuned = pipeline.fit(X, y)


In [ ]:
# check the start time
start_time = datetime.now()

# doing cross validation on the chunks of data and calculating scores
scores_rf_tuned = cross_validate(pipeline_rf_tuned, X, y, cv=time_split,
                         scoring=['neg_mean_squared_error', 'neg_mean_absolute_error',
                                  'neg_mean_squared_log_error'],
                         return_train_score=True, n_jobs=-1)

# print the total running time
end_time = datetime.now()
print('Total cross validation time for random forests:', (end_time-start_time).total_seconds())


In [ ]:
# root mean squared error
print('Random Forests: Average RMSE train data:', 
      sum([np.sqrt(-1 * x) for x in scores_rf_tuned['train_neg_mean_squared_error']])/len(scores_rf_tuned['train_neg_mean_squared_error']))
print('Random Forests: Average RMSE test data:', 
      sum([np.sqrt(-1 * x) for x in scores_rf_tuned['test_neg_mean_squared_error']])/len(scores_rf_tuned['test_neg_mean_squared_error']))

# mean absolute error
print('Random Forests: Average MAE train data:', 
      sum([(-1 * x) for x in scores_rf_tuned['train_neg_mean_absolute_error']])/len(scores_rf_tuned['train_neg_mean_absolute_error']))
print('Random Forests: Average MAE test data:', 
      sum([(-1 * x) for x in scores_rf_tuned['test_neg_mean_absolute_error']])/len(scores_rf_tuned['test_neg_mean_absolute_error']))

# root mean squared log error
print('Random Forests: Average RMSLE train data:', 
      sum([(-1 * x) for x in scores_rf_tuned['train_neg_mean_squared_log_error']])/len(scores_rf_tuned['train_neg_mean_squared_log_error']))
print('Random Forests: Average RMSLE test data:', 
      sum([(-1 * x) for x in scores_rf_tuned['test_neg_mean_squared_log_error']])/len(scores_rf_tuned['test_neg_mean_squared_log_error']))



In [ ]:
# plot feature importance
plt.figure(figsize=[12,9])
importances = list(pipeline_rf_tuned.steps[1][1].feature_importances_)

imp_dict = {key: val for key, val in zip(list(X.columns), importances)}
sorted_feats = sorted(imp_dict.items(), key=lambda x:x[1], reverse=True)
x_val = [x[0] for x in sorted_feats]
y_val = [x[1] for x in sorted_feats]

sb.barplot(y_val, x_val)


##### running model with differenced y

In [ ]:
# get the final scores for the differenced data
split = 10
all_scores_test = []
all_scores_train = []

for i in range(split):
    scores_test = split_predict_test(X_train_over, y_train_over, X_test_over, y_test_over, i)
    all_scores_test.append(scores_test)
    scores_train = split_predict_train(X_train_over, y_train_over, X_test_over, y_test_over, i)
    all_scores_train.append(scores_train)
    
rmse_test = []
rmsle_test = []
mae_test = []
rmse_train = []
rmsle_train = []
mae_train = []

for vals in all_scores_test:
    rmse_test.append(vals[0])
    rmsle_test.append(vals[1])
    mae_test.append(vals[2])
    
for vals in all_scores_train:
    rmse_train.append(vals[0])
    rmsle_train.append(vals[1])
    mae_train.append(vals[2])
    
print('Overall Test RMSE:', sum(rmse_test)/split)
print('Overall Test RMSLE:', sum(rmsle_test)/split)
print('Overall Test MAE:', sum(mae_test)/split)

print('Overall Train RMSE:', sum(rmse_train)/split)
print('Overall Train RMSLE:', sum(rmsle_train)/split)
print('Overall Train MAE:', sum(mae_train)/split)
    

##### Removing some features based on feature importance

In [ ]:
# how many features are in the dataset currently:
len(x_val)
weekday_feats = ['weekday_3', 'weekday_1', 'weekday_2', 'weekday_4', 'weekday_5', 'weekday_6']

In [ ]:
# check how the model performance without certain features that are very unimportant
X_feat_imp = X.drop(columns=weekday_feats)


In [ ]:
# creating and fitting the ML pipeline
trend_features = ['Trip_Count_mean'+str(window), 'Trip_Count_std'+str(window)]#, 'Trip_Count_t-1']

preprocessor = ColumnTransformer([
    ('trend_diff', FunctionTransformer(np.log1p, validate=False), trend_features),
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    #('debug', Debug()) # I have commented this out because it will hinder the execution of this pipeline
    ('model', model_rf_tuned)
])

# fitting x and y to pipeline
pipeline_rf_tuned = pipeline.fit(X_feat_imp, y)


In [ ]:
# doing cross validation on the chunks of data and calculating scores
scores_rf_tuned = cross_validate(pipeline_rf_tuned, X_feat_imp, y, cv=time_split,
                         scoring=['neg_mean_squared_error', 'neg_mean_absolute_error',
                                  'neg_mean_squared_log_error'],
                         return_train_score=True, n_jobs=-1)


In [ ]:
# root mean squared error
print('Random Forests: Average RMSE train data:', 
      sum([np.sqrt(-1 * x) for x in scores_rf_tuned['train_neg_mean_squared_error']])/len(scores_rf_tuned['train_neg_mean_squared_error']))
print('Random Forests: Average RMSLE test data:', 
      sum([np.sqrt(-1 * x) for x in scores_rf_tuned['test_neg_mean_squared_error']])/len(scores_rf_tuned['test_neg_mean_squared_error']))

# mean absolute error
print('Random Forests: Average MAE train data:', 
      sum([(-1 * x) for x in scores_rf_tuned['train_neg_mean_absolute_error']])/len(scores_rf_tuned['train_neg_mean_absolute_error']))
print('Random Forests: Average MAE test data:', 
      sum([(-1 * x) for x in scores_rf_tuned['test_neg_mean_absolute_error']])/len(scores_rf_tuned['test_neg_mean_absolute_error']))

# root mean squared log error
print('Random Forests: Average RMSLE train data:', 
      sum([(-1 * x) for x in scores_rf_tuned['train_neg_mean_squared_log_error']])/len(scores_rf_tuned['train_neg_mean_squared_log_error']))
print('Random Forests: Average RMSLE test data:', 
      sum([(-1 * x) for x in scores_rf_tuned['test_neg_mean_squared_log_error']])/len(scores_rf_tuned['test_neg_mean_squared_log_error']))



Removing seemingly unimportant features does not lead to a better result. Unimportant features are simply not considered.

#### 4.2. AdaBoost

##### running AdaBoost with undifference y

In [ ]:
# initializing AdaBoost model with default hyperparameters
model_ada = AdaBoostRegressor(random_state=42, base_estimator=DecisionTreeRegressor())

# creating and fitting the ML pipeline
trend_features = ['Trip_Count_mean'+str(window), 'Trip_Count_std'+str(window), 'Trip_Count_t-1']

preprocessor = ColumnTransformer([
    ('trend_diff', FunctionTransformer(np.log1p, validate=False), trend_features),
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    #('debug', Debug()) # I have commented this out because it will hinder the execution of this pipeline
    ('model', model_ada)
])

pipeline_ada = pipeline.fit(X, y)


In [ ]:
# creating a timeseries split of the datasets
time_split = TimeSeriesSplit(n_splits=10)

# doing cross validation on the chunks of data and calculating scores
scores_ada = cross_validate(pipeline_ada, X, y, cv=time_split,
                         scoring=['neg_mean_squared_error', 'neg_mean_absolute_error',
                                  'neg_mean_squared_log_error'],
                         return_train_score=True, n_jobs=-1)


In [ ]:
# root mean squared error
print('AdaBoost: Average RMSE train data:', 
      sum([np.sqrt(-1 * x) for x in scores_ada['train_neg_mean_squared_error']])/len(scores_ada['train_neg_mean_squared_error']))
print('AdaBoost: Average RMSE test data:', 
      sum([np.sqrt(-1 * x) for x in scores_ada['test_neg_mean_squared_error']])/len(scores_ada['test_neg_mean_squared_error']))

# mean absolute error
print('AdaBoost: Average MAE train data:', 
      sum([(-1 * x) for x in scores_ada['train_neg_mean_absolute_error']])/len(scores_ada['train_neg_mean_absolute_error']))
print('AdaBoost: Average MAE test data:', 
      sum([(-1 * x) for x in scores_ada['test_neg_mean_absolute_error']])/len(scores_ada['test_neg_mean_absolute_error']))

# root mean squared log error
print('AdaBoost: Average RMSLE train data:', 
      sum([np.sqrt(-1 * x) for x in scores_ada['train_neg_mean_squared_log_error']])/len(scores_ada['train_neg_mean_squared_log_error']))
print('AdaBoost: Average RMSLE test data:', 
      sum([np.sqrt(-1 * x) for x in scores_ada['test_neg_mean_squared_log_error']])/len(scores_ada['test_neg_mean_squared_log_error']))


Tuning the hyperparameters of AdaBoost

In [ ]:
# number of estimators in AdaBoost model
n_estimators = randint(100, 1000)
# learning rate
learning_rate = uniform(0.001, 0.05)
# max_depth
max_depth = randint(1, 8)

# Create the random grid
random_grid_ada = {'model__n_estimators': n_estimators,
                   'model__learning_rate': learning_rate,
                   'model__base_estimator__max_depth': max_depth}


In [ ]:
# check the start time
start_time = datetime.now()
print(start_time)

# instantiate and fit the randomizedsearch class to the random parameters
rs_ada = RandomizedSearchCV(pipeline_ada, 
                        param_distributions=random_grid_ada, 
                        scoring='neg_mean_squared_error', 
                        n_jobs=-1,
                        cv=time_split,
                        n_iter = 5,
                        verbose=10,
                        random_state=40)
rs_ada = rs_ada.fit(X, y)

# print the total running time
end_time = datetime.now()
print('Total running time:', end_time-start_time)
    

In [ ]:
# Saving the best RandomForest model
pickle.dump(rs_ada.best_estimator_, open('adaboost.sav', 'wb'))

# change the keys of the best_params dictionary to allow it to be used for the final model
fix_best_params_ada = {key[7:]: val for key, val in rs_ada.best_params_.items()}

print(fix_best_params_ada)
max_depth = fix_best_params_ada['base_estimator__max_depth']

# fit the randomforestregressor with the best_params as hyperparameters
model_ada_tuned = AdaBoostRegressor(DecisionTreeRegressor(max_depth=max_depth),
                                    learning_rate=fix_best_params_ada['learning_rate'], 
                                    n_estimators=fix_best_params_ada['n_estimators'])


In [ ]:
# creating and fitting the ML pipeline
trend_features = ['Trip_Count_mean'+str(window), 'Trip_Count_std'+str(window), 'Trip_Count_t-1']

preprocessor = ColumnTransformer([
    ('trend_diff', FunctionTransformer(np.log1p, validate=False), trend_features),
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    #('debug', Debug()) # I have commented this out because it will hinder the execution of this pipeline
    ('model', model_ada_tuned)
])

pipeline_ada_tuned = pipeline.fit(X, y)


In [ ]:
# creating a timeseries split of the datasets
time_split = TimeSeriesSplit(n_splits=10)

# doing cross validation on the chunks of data and calculating scores
scores_ada_tuned = cross_validate(pipeline_ada_tuned, X, y, cv=time_split,
                         scoring=['neg_mean_squared_error', 'neg_mean_absolute_error',
                                  'neg_mean_squared_log_error'],
                         return_train_score=True, n_jobs=-1)


In [ ]:
# root mean squared error
print('AdaBoost: Average RMSE train data:', 
      sum([np.sqrt(-1 * x) for x in scores_ada_tuned['train_neg_mean_squared_error']])/len(scores_ada_tuned['train_neg_mean_squared_error']))
print('AdaBoost: Average RMSE test data:', 
      sum([np.sqrt(-1 * x) for x in scores_ada_tuned['test_neg_mean_squared_error']])/len(scores_ada_tuned['test_neg_mean_squared_error']))

# mean absolute error
print('AdaBoost: Average MAE train data:', 
      sum([(-1 * x) for x in scores_ada_tuned['train_neg_mean_absolute_error']])/len(scores_ada_tuned['train_neg_mean_absolute_error']))
print('AdaBoost: Average MAE test data:', 
      sum([(-1 * x) for x in scores_ada_tuned['test_neg_mean_absolute_error']])/len(scores_ada_tuned['test_neg_mean_absolute_error']))

# root mean squared log error
print('AdaBoost: Average RMSLE train data:', 
      sum([np.sqrt(-1 * x) for x in scores_ada_tuned['train_neg_mean_squared_log_error']])/len(scores_ada_tuned['train_neg_mean_squared_log_error']))
print('AdaBoost: Average RMSLEtest data:', 
      sum([np.sqrt(-1 * x) for x in scores_ada_tuned['test_neg_mean_squared_log_error']])/len(scores_ada_tuned['test_neg_mean_squared_log_error']))


In [ ]:
# specify some values for hyperparameters around the values chosen by randomizedsearch
grid_param_ada = {'model__learning_rate': [0.003, 0.004, 0.0332],
                  'model__n_estimators': [100, 265],
                  'model__base_estimator__max_depth': [5, 6, 7]}


# use gridsearch to search around the randomizedsearch best parameters and further improve the model
gs_ada = GridSearchCV(pipeline_ada, 
                  param_grid=grid_param_ada, 
                  scoring='neg_mean_squared_error',
                  verbose = 10,
                  n_jobs=-1, 
                  cv=time_split)

gs_ada = gs_ada.fit(X, y)


In [ ]:
# Saving the best RandomForest model
#pickle.dump(gs_ada.best_estimator_, open('adaboost.sav', 'wb'))

# change the keys of the best_params dictionary to allow it to be used for the final model
fix_best_params_ada = {key[7:]: val for key, val in gs_ada.best_params_.items()}

print(fix_best_params_ada)

max_depth = fix_best_params_ada['base_estimator__max_depth']

# fit the randomforestregressor with the best_params as hyperparameters
model_ada_tuned = AdaBoostRegressor(DecisionTreeRegressor(max_depth=max_depth),
                                    learning_rate=fix_best_params_ada['learning_rate'], 
                                    n_estimators=fix_best_params_ada['n_estimators'])


In [ ]:
# creating and fitting the ML pipeline
trend_features = ['Trip_Count_mean'+str(window), 'Trip_Count_std'+str(window), 'Trip_Count_t-1']

preprocessor = ColumnTransformer([
    ('trend_diff', FunctionTransformer(np.log1p, validate=False), trend_features),
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    #('debug', Debug()) # I have commented this out because it will hinder the execution of this pipeline
    ('model', model_ada_tuned)
])

pipeline_ada_tuned = pipeline.fit(X, y)


In [ ]:
# creating a timeseries split of the datasets
time_split = TimeSeriesSplit(n_splits=10)

# check the start time
start_time = datetime.now()

# doing cross validation on the chunks of data and calculating scores
scores_ada_tuned = cross_validate(pipeline_ada_tuned, X, y, cv=time_split,
                         scoring=['neg_mean_squared_error', 'neg_mean_absolute_error',
                                  'neg_mean_squared_log_error'],
                         return_train_score=True, n_jobs=-1)

# print the total running time
end_time = datetime.now()
print('Total cross validation time for AdaBoost:', (end_time-start_time).total_seconds())


In [ ]:
# root mean squared error
print('AdaBoost: Average RMSE train data:', 
      sum([np.sqrt(-1 * x) for x in scores_ada_tuned['train_neg_mean_squared_error']])/len(scores_ada_tuned['train_neg_mean_squared_error']))
print('AdaBoost: Average RMSE test data:', 
      sum([np.sqrt(-1 * x) for x in scores_ada_tuned['test_neg_mean_squared_error']])/len(scores_ada_tuned['test_neg_mean_squared_error']))

# mean average error
print('AdaBoost: Average MAE train data:', 
      sum([(-1 * x) for x in scores_ada_tuned['train_neg_mean_absolute_error']])/len(scores_ada_tuned['train_neg_mean_absolute_error']))
print('AdaBoost: Average MAE test data:', 
      sum([(-1 * x) for x in scores_ada_tuned['test_neg_mean_absolute_error']])/len(scores_ada_tuned['test_neg_mean_absolute_error']))

# root mean squared log error
print('AdaBoost: Average RMSLE train data:', 
      sum([np.sqrt(-1 * x) for x in scores_ada_tuned['train_neg_mean_squared_log_error']])/len(scores_ada_tuned['train_neg_mean_squared_log_error']))
print('AdaBoost: Average RMSLE test data:', 
      sum([np.sqrt(-1 * x) for x in scores_ada_tuned['test_neg_mean_squared_log_error']])/len(scores_ada_tuned['test_neg_mean_squared_log_error']))



In [ ]:
# plot feature importance
plt.figure(figsize=[12,9])
importances = list(pipeline_ada_tuned.steps[1][1].feature_importances_)

imp_dict = {key: val for key, val in zip(list(X.columns), importances)}
sorted_feats = sorted(imp_dict.items(), key=lambda x:x[1], reverse=True)
x_val = [x[0] for x in sorted_feats]
y_val = [x[1] for x in sorted_feats]

#importances.sort(reverse=True)
sb.barplot(y_val, x_val)
#importances

##### running AdaBoost with differenced y

In [ ]:
# get the final scores for the differenced data
split = 10
all_scores_test = []
all_scores_train = []

for i in range(split):
    scores_test = split_predict_test(X_train_over, y_train_over, X_test_over, y_test_over, i)
    all_scores_test.append(scores_test)
    scores_train = split_predict_train(X_train_over, y_train_over, X_test_over, y_test_over, i)
    all_scores_train.append(scores_train)
    
rmse_test = []
rmsle_test = []
mae_test = []
rmse_train = []
rmsle_train = []
mae_train = []

for vals in all_scores_test:
    rmse_test.append(vals[0])
    rmsle_test.append(vals[1])
    mae_test.append(vals[2])
    
for vals in all_scores_train:
    rmse_train.append(vals[0])
    rmsle_train.append(vals[1])
    mae_train.append(vals[2])
    
print('Overall Test RMSE:', sum(rmse_test)/split)
print('Overall Test RMSLE:', sum(rmsle_test)/split)
print('Overall Test MAE:', sum(mae_test)/split)

print('Overall Train RMSE:', sum(rmse_train)/split)
print('Overall Train RMSLE:', sum(rmsle_train)/split)
print('Overall Train MAE:', sum(mae_train)/split)
    

##### checking how certain features influence the overall performance

In [ ]:
# check how the model performance without certain features that are very unimportant
X_feat_imp = X.drop(columns=['weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6'])


In [ ]:
# creating and fitting the ML pipeline
trend_features = ['Trip_Count_mean'+str(window), 'Trip_Count_std'+str(window)]#, 'Trip_Count_t-1']

preprocessor = ColumnTransformer([
    ('trend_diff', FunctionTransformer(np.log1p, validate=False), trend_features),
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    #('debug', Debug()) # I have commented this out because it will hinder the execution of this pipeline
    ('model', model_ada_tuned)
])

pipeline_ada_tuned = pipeline.fit(X_feat_imp, y)


In [ ]:
# creating a timeseries split of the datasets
time_split = TimeSeriesSplit(n_splits=10)

# doing cross validation on the chunks of data and calculating scores
scores_ada_tuned = cross_validate(pipeline_ada_tuned, X_feat_imp, y, cv=time_split,
                         scoring=['neg_mean_squared_error', 'neg_mean_absolute_error',
                                  'neg_mean_squared_log_error'],
                         return_train_score=True, n_jobs=-1)


In [ ]:
# root mean squared error
print('AdaBoost: Average RMSE train data:', 
      sum([np.sqrt(-1 * x) for x in scores_ada_tuned['train_neg_mean_squared_error']])/len(scores_ada_tuned['train_neg_mean_squared_error']))
print('AdaBoost: Average RMSE test data:', 
      sum([np.sqrt(-1 * x) for x in scores_ada_tuned['test_neg_mean_squared_error']])/len(scores_ada_tuned['test_neg_mean_squared_error']))

# mean average error
print('AdaBoost: Average MAE train data:', 
      sum([(-1 * x) for x in scores_ada_tuned['train_neg_mean_absolute_error']])/len(scores_ada_tuned['train_neg_mean_absolute_error']))
print('AdaBoost: Average MAE test data:', 
      sum([(-1 * x) for x in scores_ada_tuned['test_neg_mean_absolute_error']])/len(scores_ada_tuned['test_neg_mean_absolute_error']))

# root mean squared log error
print('AdaBoost: Average RMSLE train data:', 
      sum([(-1 * x) for x in scores_ada_tuned['train_neg_mean_squared_log_error']])/len(scores_ada_tuned['train_neg_mean_squared_log_error']))
print('AdaBoost: Average RMSLE test data:', 
      sum([(-1 * x) for x in scores_ada_tuned['test_neg_mean_squared_log_error']])/len(scores_ada_tuned['test_neg_mean_squared_log_error']))



#### 4.3. XGBoost

##### running XGBoost with undifferenced y

In [ ]:
# initializing XGBoost model with default hyperparameters
model_xgb = xgb.XGBRegressor(random_state=42)

# creating and fitting the ML pipeline
trend_features = ['Trip_Count_mean'+str(window), 'Trip_Count_std'+str(window), 'Trip_Count_t-1']

preprocessor = ColumnTransformer([
    ('trend_diff', FunctionTransformer(np.log1p, validate=False), trend_features),
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', MinMaxScaler()),
    #('debug', Debug()) # I have commented this out because it will hinder the execution of this pipeline
    ('model', model_xgb)
])

pipeline_xgb = pipeline.fit(X, y)


In [ ]:
# creating a timeseries split of the datasets
time_split = TimeSeriesSplit(n_splits=10)

# doing cross validation on the chunks of data and calculating scores
scores_xgb = cross_validate(pipeline_xgb, X, y, cv=time_split,
                         scoring=['neg_mean_squared_error', 'neg_mean_absolute_error',
                                  'neg_mean_squared_log_error'],
                         return_train_score=True, n_jobs=-1)


In [ ]:
# root mean squared error
print('XGBoost: Average RMSE train data:', 
      sum([np.sqrt(-1 * x) for x in scores_xgb['train_neg_mean_squared_error']])/len(scores_xgb['train_neg_mean_squared_error']))
print('XGBoost: Average RMSE test data:', 
      sum([np.sqrt(-1 * x) for x in scores_xgb['test_neg_mean_squared_error']])/len(scores_xgb['test_neg_mean_squared_error']))

# mean absolute error
print('XGBoost: Average MAE train data:', 
      sum([(-1 * x) for x in scores_xgb['train_neg_mean_absolute_error']])/len(scores_xgb['train_neg_mean_absolute_error']))
print('XGBoost: Average MAE test data:', 
      sum([(-1 * x) for x in scores_xgb['test_neg_mean_absolute_error']])/len(scores_xgb['test_neg_mean_absolute_error']))

# root mean squared log error
print('XGBoost: Average RMSLE train data:', 
      sum([np.sqrt(-1 * x) for x in scores_xgb['train_neg_mean_squared_log_error']])/len(scores_xgb['train_neg_mean_squared_log_error']))
print('XGBoost: Average RMSLE test data:', 
      sum([np.sqrt(-1 * x) for x in scores_xgb['test_neg_mean_squared_log_error']])/len(scores_xgb['test_neg_mean_squared_log_error']))


In [ ]:
# alpha
alpha = uniform(0.2, 0.5)
# learning rate
learning_rate = uniform(0, 1)
# colsample_bytree
colsample_bytree = uniform(0, 1)
# max depth
n_estimators = randint(300, 1000)
# min child weight
min_child_weight = randint(5, 10)
# max_depth
max_depth = randint(1, 5)
# subsample
# subsample = uniform(0, 1)

# Create the random grid
random_grid_xgb = {'model__n_estimators': n_estimators,
                   'model__learning_rate': learning_rate,
                   'model__reg_alpha': alpha,
                   'model__colsample_bytree': colsample_bytree,
                   'model__min_child_weight': min_child_weight,
                   'model__max_depth': max_depth}
                   #'model__subsample': subsample}


In [ ]:
# check the start time
start_time = datetime.now()
print(start_time)

# instantiate and fit the randomizedsearch class to the random parameters
rs_xgb = RandomizedSearchCV(pipeline_xgb, 
                        param_distributions=random_grid_xgb, 
                        scoring='neg_mean_squared_error', 
                        n_jobs=-1,
                        cv=time_split,
                        n_iter = 10,
                        verbose=10,
                        random_state=10)
rs_xgb = rs_xgb.fit(X, y)

# print the total running time
end_time = datetime.now()
print('Total running time:', end_time-start_time)
    

In [ ]:
# Saving the best RandomForest model
pickle.dump(rs_xgb.best_estimator_, open('xgboost.sav', 'wb'))

# change the keys of the best_params dictionary to allow it to be used for the final model
fix_best_params_xgb = {key[7:]: val for key, val in rs_xgb.best_params_.items()}

print(fix_best_params_xgb)

# fit the randomforestregressor with the best_params as hyperparameters
model_xgb_rs = xgb.XGBRegressor(**fix_best_params_xgb)


In [ ]:
# creating and fitting the ML pipeline
trend_features = ['Trip_Count_mean'+str(window), 'Trip_Count_std'+str(window), 'Trip_Count_t-1']

preprocessor = ColumnTransformer([
    ('trend_diff', FunctionTransformer(np.log1p, validate=False), trend_features),
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', MinMaxScaler()),
    #('debug', Debug()) # I have commented this out because it will hinder the execution of this pipeline
    ('model', model_xgb_rs)
])

pipeline_xgb_rs = pipeline.fit(X, y)


In [ ]:
# creating a timeseries split of the datasets
time_split = TimeSeriesSplit(n_splits=10)

# doing cross validation on the chunks of data and calculating scores
scores_xgb_rs = cross_validate(pipeline_xgb_rs, X, y, cv=time_split,
                         scoring=['neg_mean_squared_error', 'neg_mean_absolute_error',
                                  'neg_mean_squared_log_error'],
                         return_train_score=True, n_jobs=-1)


In [ ]:
# root mean squared error
print('XGBoost: Average RMSE train data:', 
      sum([np.sqrt(-1 * x) for x in scores_xgb_rs['train_neg_mean_squared_error']])/len(scores_xgb_rs['train_neg_mean_squared_error']))
print('XGBoost: Average RMSE test data:', 
      sum([np.sqrt(-1 * x) for x in scores_xgb_rs['test_neg_mean_squared_error']])/len(scores_xgb_rs['test_neg_mean_squared_error']))

# mean absolute error
print('XGBoost: Average MAE train data:', 
      sum([(-1 * x) for x in scores_xgb_rs['train_neg_mean_absolute_error']])/len(scores_xgb_rs['train_neg_mean_absolute_error']))
print('XGBoost: Average MAE test data:', 
      sum([(-1 * x) for x in scores_xgb_rs['test_neg_mean_absolute_error']])/len(scores_xgb_rs['test_neg_mean_absolute_error']))

# root mean squared log error
print('XGBoost: Average RMSLE train data:', 
      sum([np.sqrt(-1 * x) for x in scores_xgb_rs['train_neg_mean_squared_log_error']])/len(scores_xgb_rs['train_neg_mean_squared_log_error']))
print('XGBoost: Average RMSLE test data:', 
      sum([np.sqrt(-1 * x) for x in scores_xgb_rs['test_neg_mean_squared_log_error']])/len(scores_xgb_rs['test_neg_mean_squared_log_error']))



In [ ]:
# specify some values for hyperparameters around the values chosen by randomizedsearch
grid_param_xgb = {'model__n_estimators': [664, 650],
                   'model__learning_rate': [0.35, 0.04],
                   'model__alpha': [0.636, 0.6],
                   'model__colsample_bytree': [0.85, 0.9],
                   'model__min_child_weight': [7, 8],
                   'model__max_depth': [1, 2]}
                   #'model__subsample': [0.98, 0.90]}

# use gridsearch to search around the randomizedsearch best parameters and further improve the model
gs_xgb = GridSearchCV(pipeline_xgb, 
                  param_grid=grid_param_xgb, 
                  scoring='neg_mean_squared_error', 
                  verbose = 10,
                  n_jobs=-1, 
                  cv=time_split)

gs_xgb = gs_xgb.fit(X, y)


In [ ]:
# Saving the best XGB model
pickle.dump(gs_xgb.best_estimator_, open('xgboost.sav', 'wb'))

# change the keys of the best_params dictionary to allow it to be used for the final model
fix_best_params_xgb = {key[7:]: val for key, val in gs_xgb.best_params_.items()}

print(fix_best_params_xgb)

# fit the randomforestregressor with the best_params as hyperparameters
model_xgb_tuned = xgb.XGBRegressor(**fix_best_params_xgb)


In [ ]:
# creating and fitting the ML pipeline
trend_features = ['Trip_Count_mean'+str(window), 'Trip_Count_std'+str(window), 'Trip_Count_t-1']

preprocessor = ColumnTransformer([
    ('trend_diff', FunctionTransformer(np.log1p, validate=False), trend_features),
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', MinMaxScaler()),
    #('debug', Debug()) # I have commented this out because it will hinder the execution of this pipeline
    ('model', model_xgb_tuned)
])

pipeline_xgb_tuned = pipeline.fit(X, y)


In [ ]:
# creating a timeseries split of the datasets
time_split = TimeSeriesSplit(n_splits=10)

# check the start time
start_time = datetime.now()

# doing cross validation on the chunks of data and calculating scores
scores_xgb_tuned = cross_validate(pipeline_xgb_tuned, X, y, cv=time_split,
                         scoring=['neg_mean_squared_error', 'neg_mean_absolute_error',
                                  'neg_mean_squared_log_error'],
                         return_train_score=True, n_jobs=-1)

# print the total running time
end_time = datetime.now()
print('Total cross validation time for XGBBoost:', (end_time-start_time).total_seconds())


In [ ]:
# root mean squared error
print('XGBoost: Average RMSE train data:', 
      sum([np.sqrt(-1 * x) for x in scores_xgb_tuned['train_neg_mean_squared_error']])/len(scores_xgb_tuned['train_neg_mean_squared_error']))
print('XGBoost: Average RMSE test data:', 
      sum([np.sqrt(-1 * x) for x in scores_xgb_tuned['test_neg_mean_squared_error']])/len(scores_xgb_tuned['test_neg_mean_squared_error']))

# mean absolute error
print('XGBoost: Average MAE train data:', 
      sum([(-1 * x) for x in scores_xgb_tuned['train_neg_mean_absolute_error']])/len(scores_xgb_tuned['train_neg_mean_absolute_error']))
print('XGBoost: Average MAE test data:', 
      sum([(-1 * x) for x in scores_xgb_tuned['test_neg_mean_absolute_error']])/len(scores_xgb_tuned['test_neg_mean_absolute_error']))

# root mean squared log error
print('XGBoost: Average RMSLE train data:', 
      sum([np.sqrt(-1 * x) for x in scores_xgb_tuned['train_neg_mean_squared_log_error']])/len(scores_xgb_tuned['train_neg_mean_squared_log_error']))
print('XGBoost: Average RMSLE test data:', 
      sum([np.sqrt(-1 * x) for x in scores_xgb_tuned['test_neg_mean_squared_log_error']])/len(scores_xgb_tuned['test_neg_mean_squared_log_error']))


In [ ]:
# map the feature importances to the feature names
xgb_feats = X.columns
xgb_feats_dict = defaultdict()
count = 0

for item in xgb_feats:
    xgb_feats_dict['f'+str(count)] = item
    count += 1
    

In [ ]:
# Plot feature importance
fig, ax = plt.subplots(figsize=(20, 15))
xgb.plot_importance(model_xgb_tuned, importance_type='weight', ax=ax)

# rename the y axis labels to the actual feature names
locs, labels = plt.yticks()

new_names = []
for item in labels:
    for key, name in xgb_feats_dict.items():
        if key == item.get_text():
            new_names.append(name)

ax.set_yticklabels(new_names);
            
plt.show()


##### running XGBoost with differenced y

In [ ]:
# get the final scores for the differenced data
split = 10
all_scores_test = []
all_scores_train = []

for i in range(split):
    scores_test = split_predict_test(X_train_over, y_train_over, X_test_over, y_test_over, i)
    all_scores_test.append(scores_test)
    scores_train = split_predict_train(X_train_over, y_train_over, X_test_over, y_test_over, i)
    all_scores_train.append(scores_train)
    
rmse_test = []
rmsle_test = []
mae_test = []
rmse_train = []
rmsle_train = []
mae_train = []

for vals in all_scores_test:
    rmse_test.append(vals[0])
    rmsle_test.append(vals[1])
    mae_test.append(vals[2])
    
for vals in all_scores_train:
    rmse_train.append(vals[0])
    rmsle_train.append(vals[1])
    mae_train.append(vals[2])
    
print('Overall Test RMSE:', sum(rmse_test)/split)
print('Overall Test RMSLE:', sum(rmsle_test)/split)
print('Overall Test MAE:', sum(mae_test)/split)

print('Overall Train RMSE:', sum(rmse_train)/split)
print('Overall Train RMSLE:', sum(rmsle_train)/split)
print('Overall Train MAE:', sum(mae_train)/split)
    

##### checking how certain features influence the overall performance

In [ ]:
# check how the model performance without certain features that are very unimportant
X_feat_imp = X.drop(columns=['weekday_1', 'weekday_3', 'weekday_4', 'weekday_6',
                             'hot', 'cold', 'cool', 'warm'])


In [ ]:
# creating and fitting the ML pipeline
trend_features = ['Trip_Count_mean'+str(window), 'Trip_Count_std'+str(window), 'Trip_Count_t-1']

preprocessor = ColumnTransformer([
    ('trend_diff', FunctionTransformer(np.log1p, validate=False), trend_features),
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', MinMaxScaler()),
    #('debug', Debug()) # I have commented this out because it will hinder the execution of this pipeline
    ('model', model_xgb_tuned)
])

pipeline_xgb_tuned = pipeline.fit(X_feat_imp, y)


In [ ]:
# creating a timeseries split of the datasets
time_split = TimeSeriesSplit(n_splits=10)

# doing cross validation on the chunks of data and calculating scores
scores_xgb_tuned = cross_validate(pipeline_xgb_tuned, X_feat_imp, y, cv=time_split,
                         scoring=['neg_mean_squared_error', 'neg_mean_absolute_error',
                                  'neg_mean_squared_log_error'],
                         return_train_score=True, n_jobs=-1)


In [ ]:
# root mean squared error
print('XGBoost: Average RMSE train data:', 
      sum([np.sqrt(-1 * x) for x in scores_xgb_tuned['train_neg_mean_squared_error']])/len(scores_xgb_tuned['train_neg_mean_squared_error']))
print('XGBoost: Average RMSE test data:', 
      sum([np.sqrt(-1 * x) for x in scores_xgb_tuned['test_neg_mean_squared_error']])/len(scores_xgb_tuned['test_neg_mean_squared_error']))

# mean absolute error
print('XGBoost: Average MAE train data:', 
      sum([(-1 * x) for x in scores_xgb_tuned['train_neg_mean_absolute_error']])/len(scores_xgb_tuned['train_neg_mean_absolute_error']))
print('XGBoost: Average MAE test data:', 
      sum([(-1 * x) for x in scores_xgb_tuned['test_neg_mean_absolute_error']])/len(scores_xgb_tuned['test_neg_mean_absolute_error']))

# root mean squared log error
print('XGBoost: Average RMSLE train data:', 
      sum([np.sqrt(-1 * x) for x in scores_xgb_tuned['train_neg_mean_squared_log_error']])/len(scores_xgb_tuned['train_neg_mean_squared_log_error']))
print('XGBoost: Average RMSLE test data:', 
      sum([np.sqrt(-1 * x) for x in scores_xgb_tuned['test_neg_mean_squared_log_error']])/len(scores_xgb_tuned['test_neg_mean_squared_log_error']))
